# vue-class 和 style 绑定

在数据绑定中，一个常见需求是，将数据与元素的 class 列表，以及元素的 style 内联样式的操作绑定在一起。

由于它们都是属性(attribute)，因此我们可以使用 `v-bind` 来处理它们：只需从表达式中计算出最终的字符串。然而，处理字符串麻烦又容易出错。

为此，在使用 `v-bind` 指令来处理 `class` 和 `style` 时，Vue 对此做了特别的增强。

表达式除了可以是字符串，也能够是对象和数组。

### 与 HTML 的 class 绑定(Binding HTML Classes)